In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [44]:
"""
NSSCII - FEM.
Input to be parsed through.

SI-units to be used:
    + T in K
    + L in m
    + k in W/(mK)
    + q in W/m^2 - ad Neumann
    + P in W - ad nodal forces
"""

# Group number.
groupnr = 7

# Length in x- and y-direction.
L = 0.01

# Thickness (z-direction).
hz = 0.001

# Thermal conductivity (k=k_xx=k_yy, k_xy = 0.).
k = 326.

# Factor c for modifying thermal conductivity k for
# elements in elements_to_be_modified.
c = 30.

# Elements to be modified.
'''
elements_to_be_modified = [
                          7-15
                          25-31
                          43-47
                          61-63
                          79
                          ]
'''

# Boundary conditions. (q is acting as a heat sink!)
q = 3000000. # at y = L
T = 373. # at y = 0

In [45]:
class Node():
    def __init__(self, x, y, num):
        self.x = x
        self.y = y
        if num <= 10:
            self.T = T
        else:
            self.T = 0
        self.q = 0
        self.node_num = num

In [46]:
class Element():
    def __init__(self, n1, n2, n3, num):
        self.node1 = n1
        self.node2 = n2
        self.node3 = n3
        self.element_num = num
        
        
        self.b = np.zeros(3)
        self.b[0] = self.node2.y - self.node3.y
        self.b[1] = self.node3.y - self.node1.y
        self.b[2] = self.node1.y - self.node2.y
        
        self.c = np.zeros(3)
        self.c[0] = self.node3.x - self.node2.x
        self.c[1] = self.node1.x - self.node3.x
        self.c[2] = self.node2.x - self.node1.x
        
        self.area = (self.node1.x * self.b[0] + self.node2.x * self.b[1] + self.node3.x * self.b[2])/2
        
        self.stiffness_matrix = np.zeros([3, 3])
        for i in range(3):
            for j in range(3):
                self.stiffness_matrix[i][j] = self.b[i]*self.b[j] + self.c[j]*self.c[i]
        self.stiffness_matrix *= k/(4*self.area)*hz
        
        self.T = np.zeros((3, point_num))
        self.T[0][self.node1.node_num - 1] = 1
        self.T[1][self.node2.node_num - 1] = 1
        self.T[2][self.node3.node_num - 1] = 1
        
        self.global_part = self.T.transpose() @ self.stiffness_matrix @ self.T
        
    def vertices(self):
        return [self.node1.node_num, self.node2.node_num, self.node3.node_num]

In [47]:
m = 10
n = 10
point_num = m * n

x = np.linspace(0, L, n)
y = np.linspace(0, L, m)

grid = np.meshgrid(x, y)

In [48]:
points = []
for i in range(point_num):
    points.append(Node(grid[0].flatten()[i], grid[1].flatten()[i], i+1))

In [49]:
elements = []
j = 0
for i in range((n-1)*(m-1)*2):
    if i % 2 == 0:
        elements.append(Element(points[j], points[j + 1], points[j + n], i+1))
        j += n + 1
    else:
        elements.append(Element(points[j], points[j - 1], points[j - n], i+1))
        if (i + 1) % (2 * (n - 1)) == 0:
            j = j - n + 1
        else:
            j -= n

elements = []
j = 0
k = 1
for i in range(point_num-2*n+1):
    elements.append(Element(points[j], points[j+1], points[j+10], k))
    k+=1
            
    elements.append(Element(points[j+1], points[j+11], points[j+10], k))
    k+=1
    j += 1
    if (j +1) % n == 0:
        j+=1


In [50]:
glob = np.zeros((point_num, point_num))
for i in elements:
    glob += i.global_part
string =np.array2string(glob)
print(string)


[[ 0.326 -0.163  0.    ...  0.     0.     0.   ]
 [-0.163  0.652 -0.163 ...  0.     0.     0.   ]
 [ 0.    -0.163  0.652 ...  0.     0.     0.   ]
 ...
 [ 0.     0.     0.    ...  0.652 -0.163  0.   ]
 [ 0.     0.     0.    ... -0.163  0.652 -0.163]
 [ 0.     0.     0.    ...  0.    -0.163  0.326]]


In [51]:
P = np.zeros(point_num)
P[-n:] = 1/3*-q*hz*2.469135802469136e-06

In [52]:
P[-n+1:-1] *=2

In [53]:
Temp=np.ones(point_num)*T

In [54]:
Temp[10:] = np.linalg.solve(glob[10:100, 10:100], P[10:] - glob[10:100, 0:10] @ Temp[0:10])

In [55]:
P[0:10] = glob[0:10, 0:10] @ Temp[0:10] + glob[0:10, 10:100] @ Temp[10:100]


In [56]:
glob

array([[ 0.326, -0.163,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [-0.163,  0.652, -0.163, ...,  0.   ,  0.   ,  0.   ],
       [ 0.   , -0.163,  0.652, ...,  0.   ,  0.   ,  0.   ],
       ...,
       [ 0.   ,  0.   ,  0.   , ...,  0.652, -0.163,  0.   ],
       [ 0.   ,  0.   ,  0.   , ..., -0.163,  0.652, -0.163],
       [ 0.   ,  0.   ,  0.   , ...,  0.   , -0.163,  0.326]])

In [57]:
P

array([ 0.00246914,  0.00493827,  0.00493827,  0.00493827,  0.00493827,
        0.00493827,  0.00493827,  0.00493827,  0.00493827,  0.00246914,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [58]:
Temp

array([373.        , 373.        , 373.        , 373.        ,
       373.        , 373.        , 373.        , 373.        ,
       373.        , 373.        , 372.98485193, 372.98485193,
       372.98485193, 372.98485193, 372.98485193, 372.98485193,
       372.98485193, 372.98485193, 372.98485193, 372.98485193,
       372.96970386, 372.96970386, 372.96970386, 372.96970386,
       372.96970386, 372.96970386, 372.96970386, 372.96970386,
       372.96970386, 372.96970386, 372.95455578, 372.95455578,
       372.95455578, 372.95455578, 372.95455578, 372.95455578,
       372.95455578, 372.95455578, 372.95455578, 372.95455578,
       372.93940771, 372.93940771, 372.93940771, 372.93940771,
       372.93940771, 372.93940771, 372.93940771, 372.93940771,
       372.93940771, 372.93940771, 372.92425964, 372.92425964,
       372.92425964, 372.92425964, 372.92425964, 372.92425964,
       372.92425964, 372.92425964, 372.92425964, 372.92425964,
       372.90911157, 372.90911157, 372.90911157, 372.90